# Microsoft OneDrive | Microsoft OneDrive> [Microsoft OneDrive](https://en.wikipedia.org/wiki/OneDrive)（旧称 `SkyDrive`）は、Microsoftが運営するファイルホスティングサービスです。>> > [Microsoft OneDrive](https://en.wikipedia.org/wiki/OneDrive) (formerly `SkyDrive`) is a file hosting service operated by Microsoft.このノートブックでは、`OneDrive`からドキュメントを読み込む方法について説明しています。現在、サポートされているファイル形式はdocx、doc、pdfのみです。> This notebook covers how to load documents from `OneDrive`. Currently, only docx, doc, and pdf files are supported.## Prerequisites | 前提条件1. [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app)の指示に従ってアプリケーションを登録してください。   > Register an application with the [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app) instructions.2. 登録が完了すると、Azureポータルはアプリ登録の「概要」ペインを表示します。そこで「アプリケーション（クライアント）ID」を確認できます。この値は`client ID`とも呼ばれ、Microsoft identity platform内であなたのアプリケーションを一意に識別するためのものです。   > When registration finishes, the Azure portal displays the app registration's Overview pane. You see the Application (client) ID. Also called the `client ID`, this value uniquely identifies your application in the Microsoft identity platform.3. **項目1**で進める手順において、リダイレクトURIを`http://localhost:8000/callback`に設定することができます。   > During the steps you will be following at **item 1**, you can set the redirect URI as `http://localhost:8000/callback`4. **項目1**での手順を進める中で、アプリケーションシークレットセクションで新しいパスワード（`client_secret`）を生成してください。   > During the steps you will be following at **item 1**, generate a new password (`client_secret`) under Application Secrets section.5. この[ドキュメント](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope)の指示に従って、アプリケーションに以下の`SCOPES`（`offline_access`と`Files.Read.All`）を追加してください。   > Follow the instructions at this [document](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope) to add the following `SCOPES` (`offline_access` and `Files.Read.All`) to your application.6. [Graph Explorer Playground](https://developer.microsoft.com/en-us/graph/graph-explorer)にアクセスして、あなたの`OneDrive ID`を取得してください。最初のステップは、OneDriveアカウントに関連付けられているアカウントでログインしていることを確認することです。次に、`https://graph.microsoft.com/v1.0/me/drive`にリクエストを送り、レスポンスで`id`フィールドを含むペイロードが返されます。この`id`があなたのOneDriveアカウントのIDになります。   > Visit the [Graph Explorer Playground](https://developer.microsoft.com/en-us/graph/graph-explorer) to obtain your `OneDrive ID`. The first step is to ensure you are logged in with the account associated your OneDrive account. Then you need to make a request to `https://graph.microsoft.com/v1.0/me/drive` and the response will return a payload with a field `id` that holds the ID of your OneDrive account.7. `pip install o365` コマンドを使用して o365 パッケージをインストールする必要があります。   > You need to install the o365 package using the command `pip install o365`.8. 手順の最後には、以下の値を持っていなければなりません：   > At the end of the steps you must have the following values:* `CLIENT_ID`  > `CLIENT_ID`* `CLIENT_SECRET`  > `CLIENT_SECRET`* `DRIVE_ID`  > `DRIVE_ID`## 🧑 Instructions for ingesting your documents from OneDrive | 🧑 OneDriveからドキュメントを取り込むための指示### 🔑 Authentication | 🔑 認証デフォルトでは、`OneDriveLoader`は`CLIENT_ID`と`CLIENT_SECRET`の値がそれぞれ`O365_CLIENT_ID`と`O365_CLIENT_SECRET`という名前の環境変数として保存されていることを期待しています。これらの環境変数は、アプリケーションのルートにある`.env`ファイルを通じて、またはスクリプトで次のコマンドを使用して渡すことができます。> By default, the `OneDriveLoader` expects that the values of `CLIENT_ID` and `CLIENT_SECRET` must be stored as environment variables named `O365_CLIENT_ID` and `O365_CLIENT_SECRET` respectively. You could pass those environment variables through a `.env` file at the root of your application or using the following command in your script.```pythonos.environ['O365_CLIENT_ID'] = "YOUR CLIENT ID"os.environ['O365_CLIENT_SECRET'] = "YOUR CLIENT SECRET"```このローダーは、[*ユーザーに代わって*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0)と呼ばれる認証方式を使用しています。これはユーザーの同意を必要とする2段階認証です。ローダーをインスタンス化すると、アプリに必要な権限への同意をユーザーに求めるURLが表示されます。ユーザーはこのURLを訪れてアプリケーションへの同意を与える必要があります。その後、ユーザーは結果として得られるページのURLをコピーしてコンソールに貼り付けます。メソッドはログイン試行が成功した場合にTrueを返します。> This loader uses an authentication called [*on behalf of a user*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0). It is a 2 step authentication with user consent. When you instantiate the loader, it will call will print a url that the user must visit to give consent to the app on the required permissions. The user must then visit this url and give consent to the application. Then the user must copy the resulting page url and paste it back on the console. The method will then return True if the login attempt was successful.```pythonfrom langchain.document_loaders.onedrive import OneDriveLoaderloader = OneDriveLoader(drive_id="YOUR DRIVE ID")```認証が完了すると、ローダーはトークン（`o365_token.txt`）を`~/.credentials/`フォルダに保存します。このトークンは、以前説明したコピー＆ペーストの手順を踏まずに後で認証に使用できます。このトークンを認証に使用するためには、ローダーのインスタンスを生成する際に`auth_with_token`パラメータをTrueに変更する必要があります。> Once the authentication has been done, the loader will store a token (`o365_token.txt`) at `~/.credentials/` folder. This token could be used later to authenticate without the copy/paste steps explained earlier. To use this token for authentication, you need to change the `auth_with_token` parameter to True in the instantiation of the loader.```pythonfrom langchain.document_loaders.onedrive import OneDriveLoaderloader = OneDriveLoader(drive_id="YOUR DRIVE ID", auth_with_token=True)```### 🗂️ Documents loader | 🗂️ ドキュメントローダー#### 📑 Loading documents from a OneDrive Directory | 📑 OneDriveディレクトリからドキュメントを読み込む`OneDriveLoader`は、OneDrive内の特定のフォルダからドキュメントを読み込むことができます。たとえば、OneDrive内の`Documents/clients`フォルダに格納されているすべてのドキュメントを読み込みたい場合に使用できます。> `OneDriveLoader` can load documents from a specific folder within your OneDrive. For instance, you want to load all documents that are stored at `Documents/clients` folder within your OneDrive.```pythonfrom langchain.document_loaders.onedrive import OneDriveLoaderloader = OneDriveLoader(drive_id="YOUR DRIVE ID", folder_path="Documents/clients", auth_with_token=True)documents = loader.load()```#### 📑 Loading documents from a list of Documents IDs | 📑 ドキュメントIDのリストからドキュメントをロードする別の方法として、読み込みたい各ドキュメントの`object_id`のリストを提供することも可能です。そのためには、[Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer)にクエリを実行して、興味のあるすべてのドキュメントIDを見つけ出す必要があります。この[リンク](https://learn.microsoft.com/en-us/graph/api/resources/onedrive?view=graph-rest-1.0#commonly-accessed-resources)は、ドキュメントIDを取得する際に役立つエンドポイントのリストを提供しています。> Another possibility is to provide a list of `object_id` for each document you want to load. For that, you will need to query the [Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer) to find all the documents ID that you are interested in. This [link](https://learn.microsoft.com/en-us/graph/api/resources/onedrive?view=graph-rest-1.0#commonly-accessed-resources) provides a list of endpoints that will be helpful to retrieve the documents ID.例えば、Documentsフォルダのルートに保存されているすべてのオブジェクトに関する情報を取得するには、次のリクエストを行う必要があります：`https://graph.microsoft.com/v1.0/drives/{YOUR DRIVE ID}/root/children`。興味のあるIDのリストを取得したら、その後で以下のパラメータを使用してローダーをインスタンス化することができます。> For instance, to retrieve information about all objects that are stored at the root of the Documents folder, you need make a request to: `https://graph.microsoft.com/v1.0/drives/{YOUR DRIVE ID}/root/children`. Once you have the list of IDs that you are interested in, then you can instantiate the loader with the following parameters.```pythonfrom langchain.document_loaders.onedrive import OneDriveLoaderloader = OneDriveLoader(drive_id="YOUR DRIVE ID", object_ids=["ID_1", "ID_2"], auth_with_token=True)documents = loader.load()```